In [21]:
from datasets import load_dataset, concatenate_datasets
import os
import json

test_dataset = load_dataset("json",
                            data_files="hf://datasets/newfacade/LeetCodeDataset/LeetCodeDataset-v2-test-problems.jsonl")["train"]
print("Test set:", test_dataset)

train_dataset = concatenate_datasets([
    load_dataset(
        "json",
        data_files="hf://datasets/newfacade/LeetCodeDataset/LeetCodeDataset-v2-rl-problems.jsonl",
    )["train"],
    load_dataset(
        "json",
        data_files="hf://datasets/newfacade/LeetCodeDataset/LeetCodeDataset-v2-sft-problems.jsonl",
    )["train"],
]).filter(
    lambda example: example["meta"]["question_id"] not in set([d["question_id"] for d in test_dataset["meta"]]))
print("Before deduplication - Training set:", train_dataset)

Test set: Dataset({
    features: ['task_id', 'prompt', 'entry_point', 'test', 'completion', 'examples', 'src', 'meta'],
    num_rows: 200
})
Before deduplication - Training set: Dataset({
    features: ['task_id', 'prompt', 'entry_point', 'test', 'completion', 'examples', 'src', 'meta'],
    num_rows: 1890
})


In [22]:
first_time_idx = []
seen_question_ids = set()
for i, example in enumerate(train_dataset):
    if example["meta"]["question_id"] not in seen_question_ids:
        first_time_idx.append(i)
        seen_question_ids.add(example["meta"]["question_id"])
train_dataset = train_dataset.select(first_time_idx)

print("After deduplication - Training set:", train_dataset)

After deduplication - Training set: Dataset({
    features: ['task_id', 'prompt', 'entry_point', 'test', 'completion', 'examples', 'src', 'meta'],
    num_rows: 1890
})


In [28]:
dataset = []
for entry in train_dataset:
    tests = {"functional": f"{entry['test']}\n\ncheck({entry['entry_point'].strip()})"}
    problem =  f"Please solve the programming task below using a self-contained code snippet in a markdown code block.\n\n{entry['meta']['query'].strip()}"
    if len(tests) == 0:
        continue 
    new_entry = {
        "problem": problem,
        "completion": entry["completion"],
        "tests": tests,
    }
    if entry['meta']['difficulty'] != 'Easy':
        dataset.append(new_entry)

print(f'Dataset size: {len(dataset)}')

output_dir = os.path.abspath("../../train/code")
output_file = os.path.join(output_dir, "leetcode.json")

with open(output_file, "w") as f:
    json.dump(dataset, f, indent=4)

Dataset size: 1375


In [29]:
dataset = []
for entry in test_dataset:
    tests = {"functional": f"{entry['test']}\n\ncheck({entry['entry_point'].strip()})"}
    problem =  f"Please solve the programming task below using a self-contained code snippet in a markdown code block.\n\n{entry['meta']['query'].strip()}"
    if len(tests) == 0:
        continue 
    new_entry = {
        "problem": problem,
        "completion": entry["completion"],
        "tests": tests,
    }
    dataset.append(new_entry)

print(f'Dataset size: {len(dataset)}')

output_dir = os.path.abspath("../../test/code")
output_file = os.path.join(output_dir, "leetcode.json")

with open(output_file, "w") as f:
    json.dump(dataset, f, indent=4)

Dataset size: 200
